In [18]:
initial_bays = [[(3, "heavy"), ], [(1, "light"), (6, "light"),], [], [(4, "light"),  (2, "heavy"), (5, "light")], [], []]
initial_crane_position = 5
initial_crane_container_held = 0
initial_cost = 0

initial_state = ([initial_bays, initial_crane_position, initial_crane_container_held], 0)

The environment sees a set of loading_bays, where a set of containers are positioned, and a crane.
The initial state will include: - The initial set up of the containers in the bays, 
                                - The location of the crane 
                                - What container it is hoding if any

In [19]:
def is_goal_state(state):
    bays, crane_position, crane_container_held = state[0]

    # Iterate through the bays
    for i, bay in enumerate(bays):
        # Check if this bay contains containers 1, 2, and 3 in order
        if [container[0] for container in bay] == [1, 2, 3]:
            # Check if the crane is not in the same bay
            if crane_position != i:
                return True

    # If we haven't returned True by now, it's not a goal state
    return False

The goal state is defined as:  container [1, 2, 3] in any loading bay,  and the crane not in the same bay

In [20]:
def print_state(state_and_time):
    state = state_and_time[0]
    for i in range(len(state[0])):
        if i == state[1]:
            print(str(state[0][i])+" <-["+str(state[2])+"]-")
        else:
            print(str(state[0][i]))
    print("Total cost: "+str(state_and_time[1]))
    print()

this function is responsible for printing the envionment in a human friendly way

In [21]:
def perform_action(state, action):
    new_state = copy.deepcopy(state[0])  # Copy the entire state (bays, crane position, container held)

    new_state_cost = state[1]  # Track the current cost
    if action == "RIGHT":
        if new_state[1] >= (len(new_state[0]) - 1):  # If already at the rightmost bay
            return None
        new_state[1] += 1  # Move crane to the right
        cost = 3
        if new_state[2] != 0:
            if new_state[2][1] == "heavy":
                cost += 1
    elif action == "LEFT":
        if new_state[1] <= 0:  # If already at the leftmost bay
            return None
        new_state[1] -= 1  # Move crane to the left
        cost = 3
        if new_state[2] != 0:
            if new_state[2][1] == "heavy":
                cost += 1
    elif action == "DROP":

        if new_state[2] == 0:  # If no container is held
            return None
        if len([new_state[1]]) >= 4:  # Check if the current bay has already 4 containers
            return None
        cost = 5 - len([new_state[1]])  # cost based on num of containers before drop off
        container = new_state[2]
        new_state[2] = 0  # Drop the container
        new_state[0][new_state[1]].append(container)  # Add the container to the current bay
    elif action == "PICK":
        if new_state[2] != 0:  # If a container is already held
            return None
        if len(new_state[0][new_state[1]]) == 0:  # If no containers in the current bay
            return None
        cost = 5 - len([new_state[1]])  # cost based on num of containers before pick up
        container = new_state[0][new_state[1]].pop()  # Pick up the top container
        new_state[2] = container
    else:
        return None  # Invalid action

    return new_state, new_state_cost + cost



This function defines the possible action in the environment: RIGHT, LEFT, PICK, DROP.

for each action it validates it:  - checks if the crane can actualy move right or left
- chekcs if loading bay has less than 4 containers before dropping off
- checks if loading bay has at least one contaier before trying to pick up 

for each move it caluclates the cost, which is always:
- standard + 1 (for heavy container) when moving right or left
- standard - number of container in loading bay when pciking up or dropping

In [22]:
def perform_action_sequence(state, actions):
    new_state = state
    for action in actions:
        print(f"Performing action: {action}")
        result = perform_action(new_state, action)
        if result is None:  # If the action is invalid, stop processing
            print(f"Action '{action}' is invalid in the current state.")
            return None
        new_state = result  # Update the state to the result of the action
        print_state(new_state)  # Print the state after each action
    print(f"Actions taken: {actions}")
    return new_state

This function is used by the agent to apply the sequence of actions as soon as the goal is found

In [23]:
def is_action_valid(state, action):
    if perform_action(state, action):
        return True
    else:
        return False

this function is used by the agent to validate the action that it is trying to perform


In [24]:
def heuristic_function(previous_state, current_state):
    """ NOTE: we are generally decreasing estimation when state is believed to be optimal
        and increasing estimation when state is believed to not be optimal"""

    estimated_cost = 0
    # Initialize the dictionary to store bay indices for containers 1, 2, and 3
    current_bay_indices = {1: None, 2: None, 3: None}
    previous_bay_indices = {1: None, 2: None, 3: None}

    # check if containers are in crane and update bay indices according to crane position
    if current_state[0][2] != 0:
        container_id = current_state[0][2][0]
        if container_id in current_bay_indices:
            current_bay_indices[container_id] = current_state[0][1]
    if previous_state is not None and previous_state[0][2] != 0:
        container_id = previous_state[0][2][0]
        if container_id in previous_bay_indices:
            previous_bay_indices[container_id] = previous_state[0][1]

    # Loop through the rest of the bays and update the bay indices for containers 1, 2, and 3
    for i, bay in enumerate(current_state[0][0]):
        for container in bay:
            if container[0] in current_bay_indices:
                current_bay_indices[container[0]] = i
    if previous_state is not None:
        for i, bay in enumerate(previous_state[0][0]):
            for container in bay:
                if container[0] in previous_bay_indices:
                    previous_bay_indices[container[0]] = i

    # checks on container 1
    current_bay_index_1 = current_bay_indices[1]
    previous_bay_index_1 = previous_bay_indices[1]

    if current_bay_index_1 is not None:
        bay_at_index_1 = current_state[0][0][current_bay_index_1]
        # iterate through containers at 1's location
        for idx, container in enumerate(bay_at_index_1):
            if container[0] == 1:
                # increase estimation if 1 is not lowest
                if idx > 0:
                    estimated_cost += 1
                # decrease estimation if 1 is lowest
                elif idx == 0:
                    estimated_cost -= 1
                # increase estimation by num of containers above 1 if not 2
                if idx + 1 < len(bay_at_index_1):
                    if not bay_at_index_1[idx + 1][0] == 2:
                        estimated_cost += len(bay_at_index_1) - idx - 1
        # decrease cost if crane is holding cont 1 above an empty bay (good path to explore)
        if current_state[0][2] != 0 and current_state[0][2][0] == 1 and current_state[0][1] == current_bay_index_1 \
                and len(bay_at_index_1) == 0:
            estimated_cost -= 2

    # check current explored state against previous state
    if previous_bay_index_1 is not None:
        # assign previous 1's bay
        bay_at_index_1 = previous_state[0][0][previous_bay_index_1]
        containers_above_1 = []
        # find index of 1 in previous state
        for idx, container in enumerate(bay_at_index_1):
            if container[0] == 1:
                # assign all containers above 1 for use later
                containers_above_1 = bay_at_index_1[idx + 1:]
                ''' if container 1 is currently in crane, 
                    assigns estimation according to position of 1 in the previous state'''
                if current_state[0][2] != 0 and current_state[0][2][0] == 1:
                    if idx > 0:
                        estimated_cost -= 2
                    else:
                        estimated_cost += 2
        # if container picked up by crane was previously on 1 decrease cost
        if current_state[0][2] != 0:
            crane_container = current_state[0][2]
            for container in containers_above_1:
                if container == crane_container:
                    estimated_cost -= 2

    # checks on container 2
    current_bay_index_2 = current_bay_indices[2]
    previous_bay_index_2 = previous_bay_indices[2]
    if current_bay_index_2 is not None:
        # check distance to container 1
        estimated_cost += abs(current_bay_index_2 - current_bay_index_1)
        bay_at_index_2 = current_state[0][0][current_bay_index_2]
        for idx, container in enumerate(bay_at_index_2):
            if container[0] == 2:
                # check if container 2 is in the same bay as 1 but not above it
                if current_bay_index_2 == current_bay_index_1 and idx > 0 and bay_at_index_2[idx - 1][0] != 1:
                    estimated_cost += 1
                    estimated_cost += len(bay_at_index_2) - idx - 1

    # check current explored state against previous state
    if previous_bay_index_2 is not None:
        if current_state[0][2] != 0 and current_state[0][2][0] == 2:
            # checks if container 2 was in same bay as 1
            if previous_bay_index_2 == current_bay_index_1:
                bay_at_index_1 = previous_state[0][0][previous_bay_index_1]
                for idx, container in enumerate(bay_at_index_1):
                    if container[0] == 1:
                        # if container 1 was at the right position and container 2 was on top of it increase estimation
                        if idx == 0 and idx + 1 < len(bay_at_index_1) and bay_at_index_1[idx + 1][0] == 2:
                            estimated_cost += 2
                        # if container 1 was at the right position but container 2 was not on top of it decrease estimation
                        elif idx == 0 and idx + 1 < len(bay_at_index_1) and bay_at_index_1[idx + 1][0] != 2:
                            estimated_cost -= 2
                        # if container 1 was in the wrong position decrease cost
                        elif idx > 0:
                            estimated_cost -= 2
            # if container 2 was not on top of 1 decrease estimation
            elif previous_bay_index_2 != current_bay_index_1:
                estimated_cost -= 2

    # checks on container 3
    current_bay_index_3 = current_bay_indices[3]
    previous_bay_index_3 = previous_bay_indices[3]
    if current_bay_index_3 is not None:
        # check distance to container 1
        estimated_cost += abs(current_bay_index_3 - current_bay_index_1)
        bay_at_index_3 = current_state[0][0][current_bay_index_3]
        for idx, container in enumerate(bay_at_index_3):
            if container[0] == 3:
                # check if container 3 is in the same bay as 1 and not above it 2 and 1
                if current_bay_index_3 == current_bay_index_1 and idx > 0 and bay_at_index_3[idx - 1][0] != 2 \
                        and bay_at_index_3[idx - 2][0] != 1:
                    estimated_cost += 1
                    estimated_cost += len(bay_at_index_3) - idx - 1

    if previous_bay_index_3 is not None:
        # checks if the crane picked up container 3
        if current_state[0][2] != 0 and current_state[0][2][0] == 3:
            # checks if container 3 in same bay as 1 and 2
            if previous_bay_index_3 == current_bay_index_1 == current_bay_index_2:
                bay_at_index_1 = previous_state[0][0][current_bay_index_1]
                for idx, container in enumerate(bay_at_index_1):
                    if container[0] == 1:
                        # if container 1 was and cont 2 and 3 were stacked above it increase cost
                        if idx == 0 and idx + 1 < len(bay_at_index_1) and bay_at_index_1[idx + 1][0] == 2 and idx + 2 < len(bay_at_index_1) and bay_at_index_1[idx + 2][0] == 3:
                            estimated_cost += 2
                        # if container 1 was in the wrong position cont 2 and 1 were not below 3 decrease cost
                        elif idx > 0 or (idx + 1 < len(bay_at_index_1) and bay_at_index_1[idx + 1][0] != 2) or (idx + 2 < len(bay_at_index_1) and bay_at_index_1[idx + 2][0] != 3):
                            estimated_cost -= 2
            # checks if container 3 was not in same bay as 1 2
            elif previous_bay_index_3 != current_bay_index_1 or previous_bay_index_3 != current_bay_index_2:
                estimated_cost -= 2
    return estimated_cost

this is the heuristic function that will calculate h in roder to gide the agent towards making the best choice for the given state.

Generally estimation is increase when action is believed to lead further from the goal and decreased when believed to lead closer to it

the structure is as follow:   
Parameters: - previous state: it's the state on which the agent has applied the action
- current state: it's the state that was returned once the action was performed

the location of the target container is stored, wheter they are on a bay or in the crane(being moved)